In [2]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

Load the data from their combined CSV into DataFrames 

In [15]:
filename1 = '/Users/michelangelozampieri/Desktop/Repositories/Personal_Projects/Sensative_Info/Transactions/combined_AMEX.csv'
filename2 = '/Users/michelangelozampieri/Desktop/Repositories/Personal_Projects/Sensative_Info/Transactions/combined_AppleCard.csv'
filename3 = '/Users/michelangelozampieri/Desktop/Repositories/Personal_Projects/Sensative_Info/Transactions/combined_CapitalOne.csv'

In [16]:
df1 = pd.read_csv(filename1)
df2 = pd.read_csv(filename2)
df3 = pd.read_csv(filename3)

Get a sense of the data by looking at the shape, columns, and first few entries. 

In [17]:
df1.head()

,Date,Description,Amount,Extended Details,Appears On Your Statement As,Address,City/State,Zip Code,Country,Reference,Category
0,04/27/2024,MOBILE PAYMENT - THANK YOU,-100.00,MOBILE PAYMENT - THANK YOU,MOBILE PAYMENT - THANK YOU,NaN,NaN,NaN,NaN,'320241180382445031',NaN
1,04/20/2024,MOBILE PAYMENT - THANK YOU,-710.00,MOBILE PAYMENT - THANK YOU,MOBILE PAYMENT - THANK YOU,NaN,NaN,NaN,NaN,'320241110208663853',NaN
2,04/18/2024,USCIS ELIS N400 0000WASHINGTON DC,710.00,99999994110I4L06O3UU18L1K 20529\nImmigration...,USCIS ELIS N400 0000WASHINGTON DC,111 MASSACHUSETTS AVE,WASHINGTON\nDC,20529,UNITED STATES,'320241100200512051',Other-Government Services
3,04/13/2024,MOBILE PAYMENT - THANK YOU,-100.00,MOBILE PAYMENT - THANK YOU,MOBILE PAYMENT - THANK YOU,NaN,NaN,NaN,NaN,'320241040029297214',NaN
4,04/08/2024,AMAZON MARKEPLACE NA PA,27.98,2FB461MWOUL MERCHANDISE\nAMAZON MARKETPLACE NA...,AMAZON MARKETPLACE NAMZN.COM/BILL WA,410 TERRY AVE N,SEATTLE\nWA,98109,UNITED STATES,'320241000932028892',Merchandise & Supplies-Internet Purchase


In [18]:
df2.head()

,Transaction Date,Clearing Date,Description,Merchant,Category,Type,Amount (USD),Purchased By
0,09/28/2023,09/29/2023,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,Ach Deposit Internet Transfer From Account End...,Payment,Payment,-150.00,Michelangelo Zampieri
1,09/26/2023,09/27/2023,ACH DEPOSIT INTERNET TRANSFER (REVERSAL),Ach Deposit Internet Transfer (reversal),Other,Other,150.00,Michelangelo Zampieri
2,09/23/2023,09/23/2023,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,Ach Deposit Internet Transfer From Account End...,Payment,Payment,-150.00,Michelangelo Zampieri
3,09/22/2023,09/23/2023,MARKET BASKET 0000024026 MIDDLESEX TURNPIKE BU...,Market Basket,Grocery,Purchase,17.79,Michelangelo Zampieri
4,09/12/2023,09/13/2023,AMK BRANDEIS THE HOOT 415 SOUTH ST WALTHAM 024...,AMK Brandeis the Hoot,Restaurants,Purchase,11.68,Michelangelo Zampieri


In [19]:
df3.head()

,Transaction Date,Posted Date,Card No.,Description,Category,Debit,Credit
0,2023-12-28,2023-12-28,2200,CAPITAL ONE MOBILE PYMT,Payment/Credit,NaN,40.0
1,2023-12-27,2023-12-28,2200,PLAYSTATION NETWORK,Internet,19.11,NaN
2,2023-12-20,2023-12-20,2200,GOOGLE *YouTube Videos,Entertainment,24.99,NaN
3,2023-12-17,2023-12-18,2200,Spotify USA,Phone/Cable,10.99,NaN
4,2023-12-14,2023-12-15,2200,Disney Plus,Phone/Cable,24.99,NaN


In [20]:
df1.shape , df2.shape , df3.shape

((148, 11), (259, 8), (115, 7))

In [21]:
df1.columns , df2.columns , df3.columns

(Index(['Date', 'Description', 'Amount', 'Extended Details',
        'Appears On Your Statement As', 'Address', 'City/State', 'Zip Code',
        'Country', 'Reference', 'Category'],
       dtype='object'),
 Index(['Transaction Date', 'Clearing Date', 'Description', 'Merchant',
        'Category', 'Type', 'Amount (USD)', 'Purchased By'],
       dtype='object'),
 Index(['Transaction Date', 'Posted Date', 'Card No.', 'Description',
        'Category', 'Debit', 'Credit'],
       dtype='object'))

The label, what we are trying to predict is the category of the transaction, meaning that eveything else will be a feature. 

Now start cleaning the data, begin by seeing if there are any missing values. Looking at each data frames:

df1 has many missing values for the address, city and zip code, those are not really relevent so we can just drop the column altogether. 

df2 does not seem to have ny missing values 

df3 has missing values for either debit or credit, if one transaction is debit it will have null for credit and vice versa this can be fixed later. 

Start by dropping the columns not needed in df1, those will be 'address', 'state', 'zip-code', 'country', 'extended-details' and 'refrence' 

In [33]:
columns_to_drop = ['Extended Details', 'Appears On Your Statement As', 'Address', 'City/State', 'Zip Code', 'Country', 'Refrence']

columns_present = [col for col in columns_to_drop if col in df1.columns]

df1.drop(columns=columns_present, inplace=True)

In [34]:
df1.head()

,Date,Description,Amount,Category
0,04/27/2024,MOBILE PAYMENT - THANK YOU,-100.00,NaN
1,04/20/2024,MOBILE PAYMENT - THANK YOU,-710.00,NaN
2,04/18/2024,USCIS ELIS N400 0000WASHINGTON DC,710.00,Other-Government Services
3,04/13/2024,MOBILE PAYMENT - THANK YOU,-100.00,NaN
4,04/08/2024,AMAZON MARKEPLACE NA PA,27.98,Merchandise & Supplies-Internet Purchase


Now drop columns in df2 that are not needed, those being 'Clearing Date", 'Merchant', 'type' as it can be infered from the sign of the amount, and 'Purchased By'. 

In [49]:
columns_to_drop = ['Clearing Date', 'Merchant', 'Type', 'Purchased By']

columns_present = [col for col in columns_to_drop if col in df2.columns]

df2.drop(columns=columns_present, inplace=True)

In [50]:
df2.head()

,Transaction Date,Description,Category,Amount (USD)
0,09/28/2023,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,Payment,-150.00
1,09/26/2023,ACH DEPOSIT INTERNET TRANSFER (REVERSAL),Other,150.00
2,09/23/2023,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,Payment,-150.00
3,09/22/2023,MARKET BASKET 0000024026 MIDDLESEX TURNPIKE BU...,Grocery,17.79
4,09/12/2023,AMK BRANDEIS THE HOOT 415 SOUTH ST WALTHAM 024...,Restaurants,11.68


Lastly, remove any features for df3. These will be: 'Posted Date', and 'Card No.', 

Will also combine credit and debit so it will either be one or the other in a new column 'Amount', where Credit will be negative.

In [57]:
columns_to_drop = ['Posted Date', 'Card No.']

columns_present = [col for col in columns_to_drop if col in df3.columns]

df3.drop(columns=columns_present, inplace=True)

df3['Amount'] = df3['Debit'].fillna(0) + -df3['Credit'].fillna(0)


In [58]:
df3.head(10)

,Transaction Date,Description,Category,Debit,Credit,Amount
0,2023-12-28,CAPITAL ONE MOBILE PYMT,Payment/Credit,NaN,40.00,-40.00
1,2023-12-27,PLAYSTATION NETWORK,Internet,19.11,NaN,19.11
2,2023-12-20,GOOGLE *YouTube Videos,Entertainment,24.99,NaN,24.99
3,2023-12-17,Spotify USA,Phone/Cable,10.99,NaN,10.99
4,2023-12-14,Disney Plus,Phone/Cable,24.99,NaN,24.99
5,2023-12-08,APPLE.COM/BILL,Entertainment,2.99,NaN,2.99
6,2023-12-07,CAPITAL ONE AUTOPAY PYMT,Payment/Credit,NaN,38.97,-38.97
7,2023-11-17,Spotify USA,Phone/Cable,10.99,NaN,10.99
8,2023-11-14,MCDONALD'S F1473,Dining,12.97,NaN,12.97
9,2023-11-14,Disney Plus,Phone/Cable,24.99,NaN,24.99


Remove Debit and Credit features. 

In [59]:
df3 = df3.drop(columns=['Debit', 'Credit'])

In [60]:
df3.head()

,Transaction Date,Description,Category,Amount
0,2023-12-28,CAPITAL ONE MOBILE PYMT,Payment/Credit,-40.00
1,2023-12-27,PLAYSTATION NETWORK,Internet,19.11
2,2023-12-20,GOOGLE *YouTube Videos,Entertainment,24.99
3,2023-12-17,Spotify USA,Phone/Cable,10.99
4,2023-12-14,Disney Plus,Phone/Cable,24.99


In [61]:
df1.columns , df2.columns , df3.columns

(Index(['Date', 'Description', 'Amount', 'Category'], dtype='object'),
 Index(['Transaction Date', 'Description', 'Category', 'Amount (USD)'], dtype='object'),
 Index(['Transaction Date', 'Description', 'Category', 'Amount'], dtype='object'))

Now looking at all the features of the dataframes, we cans ee that they all have the same columns, just need to rename them so they have the same name and can then be combined. 